# Tweedie Compound Poisson


this is mostly about the parameterization between the Tweedie representation and the compound Poisson Gamma representation

Notation:

- lambda $\lambda$ : Poisson rate
- alpha, beta $\alpha, \beta$ : standard Gamma parameters
- tau, gamma $\tau, \gamma$ : Gamma distribution with mean and precision
- $p$ : Tweedie power parameter for variance function
- phi $\phi$ : Tweedie dispersion

E for expectation, V for variance

random variables and realization for each observation 
(typical element or vector of iid random variables, subscript i for specific observation)

- N, n : Poisson count of events (claims)
- X, x : Gamma distributed amounts for one event (size of single claim)
- Y, y : Tweedie or compound Poisson distributed total claim size


**Gamma distribution**

$E(X) = \tau = \alpha / \beta$ 

$V(X) = \tau^2 / \gamma = \alpha / \beta^2$

$\alpha = \gamma$    remark: we drop \gamma and work with \alpha in the Tweedie definition

$\beta = \gamma / \tau$

$Gamma(\alpha, \beta) = Gamma(\gamma, \gamma / \tau)$

sum of identical Gamma distributed random variables $Z = X_1 + ... + X_n$

$Z \tilde{} Gamma(n \alpha, \beta)$

$E(Z) = n E(X) = n \alpha / \beta = n \tau$

$V(Z) = n V(X) = n \alpha / \beta^2 = (n \tau)^2 / (n \gamma) = n \tau^2 / \gamma$


**Compound Poisson**

properties of compound distribution

$E(Y) = E(N) E(X)$

$V(Y) = E(N) V(X) + V(N) (E(X))^2$

for Poisson Gamma

$\mu = E(Y) = \lambda \tau$

$V(Y) = \lambda \tau^2 / \alpha + \lambda  \tau^2 = \lambda  \tau^2 (1 + 1 / \alpha)$


**Tweedie**

$E(Y) = \mu$

$V(Y) = \phi  \mu^p$

**parameter transformation compound Poisson to Tweedie**

$(\lambda, \alpha, \tau) -> (\mu, p, \phi)$

$p = (\alpha + 2) / (\alpha + 1)$   copied, derivation missing

$\mu = \lambda \tau$

$\phi = \lambda^{1 - p} \tau^{2-p} / (2 - p)$

substituting in for p, we have

$\phi = \lambda^{-1 / (1 + \alpha)} \tau^{\alpha / (1 + \alpha)} / (2 - p)$


**parameter transformation compound Poisson to Tweedie**

$(\mu, p, \phi) -> (\lambda, \alpha, \tau)$

(derived directly, I didn't find reference again, verify)

$\lambda = \mu^{2 - p} / (\phi (2 - p))$

$\tau = \mu / \lambda = \mu^{p - 1} \phi (2 - p)$

$\alpha = (2 - p) / (p - 1)$




In [1]:
import numpy as np
from scipy import stats

### Sum of Gamma random variables

helper functions for our parameterization of sum of gamma random variables

In [2]:
def get_gamma(n, mean, alpha):
	beta = alpha / mean
	return stats.gamma(n * alpha, scale=1.0 / beta)


def gamma_moments(n, mean, alpha):
	return n * mean, n * mean**2 / alpha


def gamma_transform_params(mean, alpha):
	return alpha, alpha / mean


def transform_params_t2cp(mean, dispersion, power):
	tmp = 2 - power
	p_rate = mean**tmp / dispersion / tmp  # lambda
	g_mean = mean / p_rate  # tau, mean of one gamma
	g_shape = tmp / (
		power - 1
	)  # alpha, shape parameter of gamma  #TODO check with scipy
	return p_rate, g_mean, g_shape  # lambda, tau, alpha


def transform_params_cp2t(p_rate, g_mean, g_shape):
	# lambda, tau, alpha = p_rate, g_mean, g_shape
	power = (g_shape + 2) / (g_shape + 1)  # p = (alpha + 2) / (alpha + 1)
	mean = p_rate * g_mean  # mu = lambda * tau
	# phi = lambda^{1 - p} tau^{2-p} / (2 - p)
	dispersion = p_rate ** (1 - power) * g_mean ** (2 - power) / (2 - power)
	return mean, dispersion, power

In [3]:
n, mean, alpha = 5, 10.0, 2
nobs = 50000

g = get_gamma(n, mean, alpha)
x = g.rvs(size=nobs)
print(x.mean(), x.var())
print(gamma_moments(n, mean, alpha))

49.9213085308 247.218239837
(50.0, 250.0)


In [4]:
g1 = get_gamma(1, mean, alpha)
x1 = g1.rvs(size=nobs)
print(x1.mean(), x1.var())
print(gamma_moments(1, mean, alpha))

10.0174283961 50.0381128838
(10.0, 50.0)


In [5]:
# try vectorized   # make sure we get independent random variables with broadcasting
nobs_half = nobs // 2
mean_gamma = np.repeat([5, 10], [nobs // 2, nobs - nobs // 2])
gv = get_gamma(n, mean_gamma, alpha)
xv = gv.rvs(size=nobs)
print(xv.mean(), xv.var())
print(xv[:nobs_half].mean(), xv[:nobs_half].var())
print(xv[nobs_half:].mean(), xv[nobs_half:].var())

print(gamma_moments(n, mean_gamma[[0, -1]], alpha))
print(gamma_moments(n, mean_gamma[[0, -1]].mean(), alpha), 'average tau')

37.5437058362 314.018469859
25.0339588498 62.8157470727
50.0534528226 252.233653316
(array([25, 50]), array([  62.5,  250. ]))
(37.5, 140.625) average tau


### Experiment: heterogeneity in Gamma sample

There is some additivity in Gamma distribution, even with heterogeneous alpha but common beta. But it doesn't seem to work out.
Something is missing here, i.e. I'm not comparing the right things.

try to keep beta fixed and adjust alpha

alpha = tau * beta


In [6]:
nobs_half = nobs // 2
mean_gamma = np.repeat([5, 10], [nobs // 2, nobs - nobs // 2])
# implied beta and alpha
beta = mean_gamma.mean() / alpha
alpha2 = mean_gamma / beta
n = 1
gv = get_gamma(n, mean_gamma, alpha2)
xv = gv.rvs(size=nobs)
print(xv.mean(), xv.var())
print(xv[:nobs_half].mean(), xv[:nobs_half].var())
print(xv[nobs_half:].mean(), xv[nobs_half:].var())

print(gamma_moments(n, mean_gamma[[0, -1]], alpha2[[0, -1]]))
print(gamma_moments(n, mean_gamma[[0, -1]].mean(), alpha), 'average tau')

7.48182207983 34.0763575434
4.97122808799 18.5838655488
9.99241607167 36.9626851543
(array([ 5, 10]), array([ 18.75,  37.5 ]))
(7.5, 28.125) average tau


In [7]:
alpha2[[0, -1]], alpha2[[0, -1]].mean(), alpha

(array([ 1.33333333,  2.66666667]), 2.0, 2)

In [8]:
beta

3.75

In [9]:
stats.gamma.fit(xv[:nobs_half], floc=0)

(1.3161314209771906, 0, 3.7771517408926223)

In [10]:
stats.gamma.fit(xv[nobs_half:], floc=0)

(2.6787044454247018, 0, 3.7303167539574598)

In [11]:
stats.gamma.fit(xv, floc=0)

(1.4900531123550462, 0, 5.0211781162637052)

In [12]:
gamma_transform_params(mean, alpha)

(2, 0.2)

In [13]:
gamma_transform_params(mean_gamma[[0, -1]], alpha2[[0, -1]])

(array([ 1.33333333,  2.66666667]), array([ 0.26666667,  0.26666667]))

In [14]:
alpha2[[0, -1]] / beta**2

array([ 0.09481481,  0.18962963])

In [15]:
alpha / beta**2

0.14222222222222222

## Simulate Compound Poisson Tweedie

In [16]:
# try vectorized   # make sure we get independent random variables with broadcasting
nobs = 10000
nobs_half = nobs // 2
mean_tweedie = np.repeat([5, 10], [nobs // 2, nobs - nobs // 2])
indicator = mean_tweedie == 10
dispersion = 2
power = 1.5
p_rate, g_mean, alpha = transform_params_t2cp(
	mean_tweedie, dispersion, power
)  # lambda, tau, alpha

# n is a bad choice because we use it as shortcut to number in names
y_poi = np.random.poisson(p_rate, size=nobs)

# Note gamma.rvs doesn't work with alpha=0, i.e. y_poisson = 0

mask_nonzeros = y_poi > 0
gv_ = get_gamma(y_poi[mask_nonzeros], g_mean[mask_nonzeros], alpha)
xv_ = gv_.rvs(size=mask_nonzeros.sum())
xv = np.zeros(y_poi.shape)
xv[mask_nonzeros] = xv_
print(xv.mean(), xv.var())
print(xv[:nobs_half].mean(), xv[:nobs_half].var())
print(xv[nobs_half:].mean(), xv[nobs_half:].var())

print(mean_tweedie[[0, -1]])
print(dispersion * mean_tweedie[[0, -1]] ** power)

7.59382336737 49.2819146512
5.10368166511 22.3507122165
10.0839650696 63.8115056912
[ 5 10]
[ 22.36067977  63.2455532 ]


In [17]:
def simulate_tweedie(mean, dispersion, power):
	"""
	Simulate Tweedie random variables

	Parameters
	----------
	mean_tweedie : array_like
	    array of means of Tweedie distribution
	dispersion : float
	    dispersion of Tweedie distribution
	power : float in interval (1, 2)
	    variance parameter for the Tweedie distribution

	"""
	mean = np.asarray(mean)
	p_rate, g_mean, alpha = transform_params_t2cp(
		mean, dispersion, power
	)  # lambda, tau, alpha

	# n is a bad choice because we use it as shortcut to number in names
	y_poi = np.random.poisson(p_rate, size=nobs)

	# Note gamma.rvs doesn't work with alpha=0, i.e. y_poisson = 0

	mask_nonzeros = y_poi > 0
	gv_ = get_gamma(y_poi[mask_nonzeros], g_mean[mask_nonzeros], alpha)
	xv_ = gv_.rvs(size=mask_nonzeros.sum())
	xv = np.zeros(y_poi.shape)
	xv[mask_nonzeros] = xv_
	return xv

In [18]:
# Note: we cannot simulate 0 gamma random variables
# get_gamma(0, 10, alpha).rvs()  # raises ValueError: Domain error in arguments.
# np.random.gamma(0, 10, size=5) # raises ValueError shape <= 0

In [19]:
import statsmodels.api as sm

family_link = sm.families.Tweedie(link_power=0, var_power=1.5)

xv2 = simulate_tweedie(mean_tweedie, dispersion, power)
endog = xv2
exog = sm.add_constant(indicator)

model1 = sm.GLM(endog, exog, family=family_link)
res1 = model1.fit()
print(res1.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9998
Model Family:                 Tweedie   Df Model:                            1
Link Function:                    log   Scale:                   2.05612722742
Method:                          IRLS   Log-Likelihood:                    nan
Date:                Fri, 29 Apr 2016   Deviance:                       24836.
Time:                        20:30:19   Pearson chi2:                 2.06e+04
No. Iterations:                     7                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5952      0.014    117.207      0.000       1.568       1.622
x1             0.7352      0.018     41.524      0.0

In [20]:
res1.model.estimate_tweedie_power(res1.fittedvalues), res1.scale

(1.546280648061214, 2.0561272274176154)

In [21]:
(np.exp(res1.params.cumsum()),)

(array([  4.92907119,  10.28148966]),)

In [22]:
xv.shape, indicator.shape

((10000,), (10000,))

# A brief Monte Carlo Study

This uses the true power parameter in estimating the mean parameters and dispersion. It doesn't use the estimated power parameter, which is unbiased in this case but the mean parameters don't include the extra noise. 
However, the mean parameter are also consistent with misspecified variance, because of the orthogonality between mean and variance parameters. The standard errors of the parameter estimates would be incorrect if variance is misspecified.

In [23]:
import time

result_mc = []
n_rep = 1000
t0 = time.time()
for i in range(n_rep):
	endog = simulate_tweedie(mean_tweedie, dispersion, power)
	model1 = sm.GLM(endog, exog, family=family_link)
	res1 = model1.fit()
	p, dis = res1.model.estimate_tweedie_power(res1.fittedvalues), res1.scale
	r = np.concatenate((res1.params, res1.bse, [p, dis]))
	result_mc.append(r)

t1 = time.time()
print('time', t1 - t0)
result_mc = np.asarray(result_mc)

time 41.97686195373535


In [24]:
print('MC mean and variance')
m_mc = result_mc.mean(0)
v_mc = result_mc.var(0)
print(m_mc)
print(v_mc)
print()
print('var params', m_mc[-2:], 'true:', [1.5, 2])

MC mean and variance
[ 1.60961325  0.69332718  0.01337654  0.01747699  1.49635664  2.00079205]
[  1.70755365e-04   2.88743317e-04   1.31063351e-08   2.05875134e-08
   2.25523156e-03   9.78343146e-04]

var params [ 1.49635664  2.00079205] true: [1.5, 2]


In [25]:
params_true = np.concatenate((np.log([5]), np.diff(np.log([5, 10]))))
print('mean params', m_mc[:2])
print('true params', params_true)

mean params [ 1.60961325  0.69332718]
true params [ 1.60943791  0.69314718]


In [26]:
np.log([5, 10]), np.diff(np.log([5, 10]))

(array([ 1.60943791,  2.30258509]), array([ 0.69314718]))

In [27]:
print('mean bse', result_mc[:, 2:4].mean(0))
print('MC std  ', result_mc[:, :2].std(0))

mean bse [ 0.01337654  0.01747699]
MC std   [ 0.01306734  0.01699245]


## Aggregation Property of Tweedie Compound Poisson

The underlying compound Poisson process is homogeneous in time. Tweedie is also a exponential model that satisfies several aggregation properties. In the following we look at the results of estimators when we aggregate observations. In the compound Poisson interpretation the observations are aggregates of periods of time. If we assume that the data represent one period in the initial case, then each observation in the aggregated case represents two periods. In count applications and similarly in Tweedie we can represent the underlying number of time and persion units by the exposure. In the insurance case, exposure is the total time policies are in effect that are represented by an observation. In epidemiological application, exposure commonly be in person-years.

First we look at the original data which is the last replication from our Monte Carlo run above. By construction, the data is generated by a Tweedie compound Poisson process with a relatively large sample size, therefore estimation will not be distorted by possible misspecification and the noise or randomness in the estimates will be relatively small.

Then, we combine two observations into one, where by construction observations have the same values of the explanatory variables. In this exercise the exposure increases for all information by the same amount, specifically the expectation of the response variable for each observation is now twice the original value.

Summary of results

- using exposure=2 replicates our original estimates except for small differences based on numerical optimization.
- ignoring the aggregation of observation leads to doubling of the mean parameter but does not change their standard errors.
- rescaling the response variable (dividing by 2) and using frequency weights leads to the same mean parameter estimates as in the original sample, but produces small changes in the standard errors. The response variable is now the mean of two observations, while the frequency weight indicates that this mean stands for two observations. This means that for each two observation we replaced the original value by the average of the two observation. In general, this can be capture by variance weights for distributions where taking the mean of observations is distributed by the same distribution family as the original observation.


In [29]:
print(res1.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                10000
Model:                            GLM   Df Residuals:                     9998
Model Family:                 Tweedie   Df Model:                            1
Link Function:                    log   Scale:                   1.93864085019
Method:                          IRLS   Log-Likelihood:                    nan
Date:                Fri, 29 Apr 2016   Deviance:                       23927.
Time:                        20:44:49   Pearson chi2:                 1.94e+04
No. Iterations:                     7                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5990      0.013    121.118      0.000       1.573       1.625
x1             0.7065      0.017     41.012      0.0

In [31]:
endog_year = endog.reshape(-1, 2).sum(1)
exog_year = exog[::2]

In [51]:
model_year = sm.GLM(
	endog_year, exog_year, family=family_link, exposure=2 * np.ones(endog_year.shape)
)
res_year = model_year.fit()
p_year, dis_year = (
	res_year.model.estimate_tweedie_power(res_year.fittedvalues),
	res_year.scale,
)
print(p_year, dis_year)

1.5672255066101113 1.33006176142


In [52]:
print(res_year.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                 5000
Model:                            GLM   Df Residuals:                     4998
Model Family:                 Tweedie   Df Model:                            1
Link Function:                    log   Scale:                   1.33006176142
Method:                          IRLS   Log-Likelihood:                    nan
Date:                Fri, 29 Apr 2016   Deviance:                       7406.0
Time:                        20:53:43   Pearson chi2:                 6.65e+03
No. Iterations:                     6                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5990      0.013    122.960      0.000       1.574       1.625
x1             0.7065      0.017     41.636      0.0

In [53]:
np.exp(res_year.params.cumsum())

array([  4.94828969,  10.02929738])

In [54]:
model_year = sm.GLM(
	endog_year, exog_year, family=family_link, freq_weights=np.ones(endog_year.shape)
)
res_year = model_year.fit()
p_year, dis_year = (
	res_year.model.estimate_tweedie_power(res_year.fittedvalues),
	res_year.scale,
)
print(p_year, dis_year)

1.5672255066101095 1.33006176142


In [55]:
print(res_year.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                 5000
Model:                            GLM   Df Residuals:                     4998
Model Family:                 Tweedie   Df Model:                            1
Link Function:                    log   Scale:                   1.33006176142
Method:                          IRLS   Log-Likelihood:                    nan
Date:                Fri, 29 Apr 2016   Deviance:                       7406.0
Time:                        20:53:48   Pearson chi2:                 6.65e+03
No. Iterations:                     6                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.2922      0.013    176.261      0.000       2.267       2.318
x1             0.7065      0.017     41.636      0.0

In [56]:
np.exp(res_year.params.cumsum()) / 2

array([  4.94828969,  10.02929738])

In [61]:
model_year = sm.GLM(
	endog_year / 2,
	exog_year,
	family=family_link,
	freq_weights=2 * np.ones(endog_year.shape),
)
res_year = model_year.fit()
p_year, dis_year = (
	res_year.model.estimate_tweedie_power(res_year.fittedvalues),
	res_year.scale,
)
print(p_year, dis_year)

1.5725796672896315 0.94030755413


In [62]:
print(res_year.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                 5000
Model:                            GLM   Df Residuals:                     9998
Model Family:                 Tweedie   Df Model:                            1
Link Function:                    log   Scale:                   0.94030755413
Method:                          IRLS   Log-Likelihood:                    nan
Date:                Fri, 29 Apr 2016   Deviance:                       10474.
Time:                        20:57:43   Pearson chi2:                 9.40e+03
No. Iterations:                     6                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5990      0.009    173.910      0.000       1.581       1.617
x1             0.7065      0.012     58.888      0.0

In [63]:
np.exp(res_year.params.cumsum())

array([  4.94828969,  10.02929738])